# CNN vs. ConvNN vs. ConvNN location Accuracy + Speed Test
- Now testing for removing the spatial layer with 1x1 convolution layer.

In [2]:
# Torch
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch import optim 


# Train + Data  
import sys 
sys.path.append('../Layers')
from Conv1d_NN import *
from Conv2d_NN import *

from Conv1d_NN_spatial import * 
from Conv2d_NN_spatial import * 

sys.path.append('../Data')
from CIFAR10 import * 


sys.path.append('../Train')
from train2d import * 


/Users/mingikang/miniforge3/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## I. Models

In [3]:
# Classic CNN 2D Model - Control Model 

CNN_2D = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.Flatten(),
    nn.Linear(32768, 1024),
    nn.ReLU(),
    nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary 
summary(CNN_2D, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              ReLU-2           [-1, 16, 32, 32]               0
            Conv2d-3           [-1, 32, 32, 32]           4,640
              ReLU-4           [-1, 32, 32, 32]               0
           Flatten-5                [-1, 32768]               0
            Linear-6                 [-1, 1024]      33,555,456
              ReLU-7                 [-1, 1024]               0
            Linear-8                   [-1, 10]          10,250
Total params: 33,570,794
Trainable params: 33,570,794
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.02
Params size (MB): 128.06
Estimated Total Size (MB): 129.09
----------------------------------------------------------------


In [4]:
# ConNN 2D All Sample Model with K = 9
Conv2dNN_9_all = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_all, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 12, 256]               0
            Conv1d-2              [-1, 64, 256]           6,976
         Conv1d_NN-3              [-1, 64, 256]               0
         Conv2d_NN-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 64, 256]               0
            Conv1d-7             [-1, 128, 256]          73,856
         Conv1d_NN-8             [-1, 128, 256]               0
         Conv2d_NN-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

In [5]:
# ConNN 2D Random Sample n = 64 Model with K = 9
Conv2dNN_9_64 = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=64, 
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=64, 
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_64, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 12, 256]               0
            Conv1d-2              [-1, 64, 256]           6,976
         Conv1d_NN-3              [-1, 64, 256]               0
         Conv2d_NN-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 64, 256]               0
            Conv1d-7             [-1, 128, 256]          73,856
         Conv1d_NN-8             [-1, 128, 256]               0
         Conv2d_NN-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

In [6]:
# ConNN 2D Spatial Sample n = 8 Model with K = 9
Conv2dNN_spatial_9_8 = nn.Sequential(
   Conv2d_NN_spatial(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=8, 
   ), 
   nn.ReLU(),

   Conv2d_NN_spatial(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=8, 
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_spatial_9_8, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 12, 256]               0
            Conv1d-2              [-1, 64, 256]           6,976
 Conv1d_NN_spatial-3              [-1, 64, 256]               0
 Conv2d_NN_spatial-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 64, 256]               0
            Conv1d-7             [-1, 128, 256]          73,856
 Conv1d_NN_spatial-8             [-1, 128, 256]               0
 Conv2d_NN_spatial-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

In [7]:
# ConNN 2D All Sample Model with K = 9 with Location Channels
Conv2dNN_9_all_location = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
      location_channels = True
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
      location_channels = True

   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_all_location, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 20, 256]               0
            Conv1d-2              [-1, 72, 256]          13,032
         Conv1d_NN-3              [-1, 72, 256]               0
            Conv2d-4           [-1, 16, 32, 32]             304
         Conv2d_NN-5           [-1, 16, 32, 32]               0
              ReLU-6           [-1, 16, 32, 32]               0
           Flatten-7              [-1, 72, 256]               0
            Conv1d-8             [-1, 136, 256]          88,264
         Conv1d_NN-9             [-1, 136, 256]               0
           Conv2d-10           [-1, 32, 32, 32]           1,120
        Conv2d_NN-11           [-1, 32, 32, 32]               0
             ReLU-12           [-1, 32, 32, 32]               0
          Flatten-13                [-1, 32768]               0
           Linear-14                 [-

In [8]:
# ConNN 2D Random Sample n = 64 Model with K = 9 and Location Channels
Conv2dNN_9_64_location = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=64, 
      location_channels = True
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=64, 
      location_channels = True
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_64_location, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 20, 256]               0
            Conv1d-2              [-1, 72, 256]          13,032
         Conv1d_NN-3              [-1, 72, 256]               0
            Conv2d-4           [-1, 16, 32, 32]             304
         Conv2d_NN-5           [-1, 16, 32, 32]               0
              ReLU-6           [-1, 16, 32, 32]               0
           Flatten-7              [-1, 72, 256]               0
            Conv1d-8             [-1, 136, 256]          88,264
         Conv1d_NN-9             [-1, 136, 256]               0
           Conv2d-10           [-1, 32, 32, 32]           1,120
        Conv2d_NN-11           [-1, 32, 32, 32]               0
             ReLU-12           [-1, 32, 32, 32]               0
          Flatten-13                [-1, 32768]               0
           Linear-14                 [-

In [9]:
# ConNN 2D Spatial Sample n = 8 Model with K = 9 and Location Channels
Conv2dNN_spatial_9_8_location = nn.Sequential(
   Conv2d_NN_spatial(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=8, 
      location_channels = True
   ), 
   nn.ReLU(),

   Conv2d_NN_spatial(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=8, 
      location_channels = True
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_spatial_9_8_location, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 20, 256]               0
            Conv1d-2              [-1, 72, 256]          13,032
 Conv1d_NN_spatial-3              [-1, 72, 256]               0
            Conv2d-4           [-1, 16, 32, 32]             304
 Conv2d_NN_spatial-5           [-1, 16, 32, 32]               0
              ReLU-6           [-1, 16, 32, 32]               0
           Flatten-7              [-1, 72, 256]               0
            Conv1d-8             [-1, 136, 256]          88,264
 Conv1d_NN_spatial-9             [-1, 136, 256]               0
           Conv2d-10           [-1, 32, 32, 32]           1,120
Conv2d_NN_spatial-11           [-1, 32, 32, 32]               0
             ReLU-12           [-1, 32, 32, 32]               0
          Flatten-13                [-1, 32768]               0
           Linear-14                 [-

## II. Training

In [10]:
# CIFAR10
cifar10 = CIFAR10()

Files already downloaded and verified
Files already downloaded and verified


In [10]:
# Classic CNN 2D Model - Control Model 
CNN_2D.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_2D.parameters(), lr=0.001)
num_epochs = 10 
train_model(CNN_2D, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(CNN_2D, cifar10.test_loader)

Epoch 1, Time: 21.162396907806396, Loss: 1.3826089839801154
Epoch 2, Time: 20.143177032470703, Loss: 0.852757565277007
Epoch 3, Time: 20.083405017852783, Loss: 0.4226437311457551
Epoch 4, Time: 19.730817079544067, Loss: 0.11825609279563055
Epoch 5, Time: 20.655309915542603, Loss: 0.0521453790357122
Epoch 6, Time: 20.64551019668579, Loss: 0.04739236570638664
Epoch 7, Time: 20.76027226448059, Loss: 0.05127583043096001
Epoch 8, Time: 20.42319107055664, Loss: 0.039755888418574366
Epoch 9, Time: 24.696635007858276, Loss: 0.03009036650071772
Epoch 10, Time: 25.13558077812195, Loss: 0.03986931302508934

 Average epoch time: 21.34362952709198
Accuracy on test set: 63.58%


63.58

In [12]:
# ConNN 2D All Sample Model with K = 9
Conv2dNN_9_all.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_all.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_all, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_all, cifar10.test_loader)

Epoch 1, Time: 53.536112785339355, Loss: 1.575473963909442
Epoch 2, Time: 53.22274422645569, Loss: 1.2543908624392945
Epoch 3, Time: 54.15564584732056, Loss: 1.089353429005884
Epoch 4, Time: 53.45938301086426, Loss: 0.9406443978362071
Epoch 5, Time: 54.58158802986145, Loss: 0.7814205540415576
Epoch 6, Time: 54.29331994056702, Loss: 0.6112388990571737
Epoch 7, Time: 54.77973794937134, Loss: 0.4645685204840682
Epoch 8, Time: 53.33805990219116, Loss: 0.3392346030782403
Epoch 9, Time: 52.909132957458496, Loss: 0.2584887896723988
Epoch 10, Time: 53.67305588722229, Loss: 0.20909986408698894

 Average epoch time: 53.79487805366516
Accuracy on test set: 55.24%


55.24

In [12]:
# ConNN 2D Random Sample n = 64 Model with K = 9
Conv2dNN_9_64.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_64.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_64, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_64, cifar10.test_loader)

Epoch 1, Time: 56.45228099822998, Loss: 1.7158299399458843
Epoch 2, Time: 55.29117798805237, Loss: 1.4238352504989984
Epoch 3, Time: 55.23336696624756, Loss: 1.266322964292658
Epoch 4, Time: 55.632041931152344, Loss: 1.1448007559837283
Epoch 5, Time: 55.56106185913086, Loss: 1.0268393658158723
Epoch 6, Time: 55.37459874153137, Loss: 0.9053238608190776
Epoch 7, Time: 55.27141976356506, Loss: 0.7859969618146682
Epoch 8, Time: 55.177175998687744, Loss: 0.671460329533538
Epoch 9, Time: 55.22374606132507, Loss: 0.5529599985503175
Epoch 10, Time: 56.643311738967896, Loss: 0.4613618638433154

 Average epoch time: 55.586018204689026
Accuracy on test set: 53.44%


53.44

In [13]:
# ConNN 2D Spatial Sample n = 8 => n^2 = 64 Model with K = 9
Conv2dNN_spatial_9_8.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_spatial_9_8.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_spatial_9_8, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_spatial_9_8, cifar10.test_loader)

Epoch 1, Time: 54.87223815917969, Loss: 1.6501899878387256
Epoch 2, Time: 54.47194480895996, Loss: 1.3243551953216952
Epoch 3, Time: 53.64143896102905, Loss: 1.1491762849376024
Epoch 4, Time: 54.389572858810425, Loss: 0.9833448914158375
Epoch 5, Time: 54.38466119766235, Loss: 0.7966888799234424
Epoch 6, Time: 55.00217795372009, Loss: 0.6007951954212944
Epoch 7, Time: 55.563284158706665, Loss: 0.42141883248639533
Epoch 8, Time: 53.779325008392334, Loss: 0.2924415491083089
Epoch 9, Time: 192.75495100021362, Loss: 0.21352661587774296
Epoch 10, Time: 54.21927571296692, Loss: 0.16952599402126448

 Average epoch time: 68.30788698196412
Accuracy on test set: 55.43%


55.43

### Location Channels added

In [11]:
# ConNN 2D All Sample Model with K = 9 with Location Channels
Conv2dNN_9_all_location.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_all_location.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_all_location, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_all_location, cifar10.test_loader)

Epoch 1, Time: 130.51426100730896, Loss: 1.5435681341554197
Epoch 2, Time: 159.21517062187195, Loss: 1.2268279402152351
Epoch 3, Time: 167.69118571281433, Loss: 1.0181220617440656
Epoch 4, Time: 156.31965899467468, Loss: 0.8077325787766815
Epoch 5, Time: 155.15347290039062, Loss: 0.5820138146124227
Epoch 6, Time: 167.01376008987427, Loss: 0.3828633892185548
Epoch 7, Time: 157.09908199310303, Loss: 0.26896879983032146
Epoch 8, Time: 157.23550176620483, Loss: 0.19026015255400133
Epoch 9, Time: 162.2837061882019, Loss: 0.14877870038408986
Epoch 10, Time: 157.19287705421448, Loss: 0.1315054105716231

 Average epoch time: 156.9718676328659
Accuracy on test set: 56.89%


56.89

In [14]:
# ConNN 2D Random Sample n = 64 Model with K = 9 with Location Channels
Conv2dNN_9_64_location.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_64_location.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_64_location, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_64_location, cifar10.test_loader)

Epoch 1, Time: 135.62935614585876, Loss: 1.6342949089796648
Epoch 2, Time: 156.46549320220947, Loss: 1.3548902600927426
Epoch 3, Time: 170.0567228794098, Loss: 1.1854932596311545
Epoch 4, Time: 162.83345103263855, Loss: 1.0073604974752801
Epoch 5, Time: 156.88570499420166, Loss: 0.833394475224073
Epoch 6, Time: 170.9777238368988, Loss: 0.6462010054484658
Epoch 7, Time: 173.9958691596985, Loss: 0.48588765843216414
Epoch 8, Time: 162.9570701122284, Loss: 0.35269703116753826
Epoch 9, Time: 174.4305329322815, Loss: 0.2621450355690916
Epoch 10, Time: 170.03324699401855, Loss: 0.20419485901799195

 Average epoch time: 163.4265171289444
Accuracy on test set: 58.29%


58.29

In [ ]:
# ConNN 2D Spatial Sample n = 8 => n^2 = 64 Model with K = 9 with Location Channels
Conv2dNN_spatial_9_8_location.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_spatial_9_8_locatiqon.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_spatial_9_8_location, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_spatial_9_8_location, cifar10.test_loader)

Epoch 1, Time: 141.58063411712646, Loss: 1.614033894313266
Epoch 2, Time: 156.56289100646973, Loss: 1.3019717042251011
Epoch 3, Time: 171.5555820465088, Loss: 1.0707860289479765
Epoch 4, Time: 171.92615509033203, Loss: 0.8400572804386354
Epoch 5, Time: 166.15003991127014, Loss: 0.583641728171912
Epoch 6, Time: 157.19238901138306, Loss: 0.35773437182464257
Epoch 7, Time: 155.74016094207764, Loss: 0.2244796627927619
Epoch 8, Time: 162.7372772693634, Loss: 0.1797437792753472
Epoch 9, Time: 161.57880401611328, Loss: 0.14452699768830976
Epoch 10, Time: 156.74216485023499, Loss: 0.12840148326440637

 Average epoch time: 160.17660982608794
Accuracy on test set: 56.21%


56.21